In [1]:
# from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index import download_loader

CJKPDFReader = download_loader("CJKPDFReader")
loader = CJKPDFReader()
documents = loader.load_data(file="pdf_data/data_1.pdf")
print(len(documents))

1


In [2]:
with open("./pdf_to_text/pdf_to_text_1.txt",encoding="utf-8",mode="w") as f:
    f.write(documents[0].text) 

In [3]:
paragraph_separator = "\n\n" # 段落分割
chunk_size = 1024 #チャンク（トークン）数
chunk_overlap = 20 # 前のチャンクをどのくらい含めるか
secondary_chunking_regex = '[^,.．;。]+[,.．;。]?'# 文分割

In [8]:
from llama_index.text_splitter import SentenceSplitter
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex,ServiceContext, LLMPredictor
from langchain.chat_models import ChatOpenAI

# ノードパーサーの準備
text_splitter = SentenceSplitter(
    paragraph_separator=paragraph_separator,
    secondary_chunking_regex = secondary_chunking_regex,
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
)
node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter
)

llm_predictor_gpt3 = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
llm_predictor_gpt4 = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4"))

# embedding_llm = LangchainEmbedding(
#     OpenAIEmbeddings(
#         model=embedding_llm_model_name,
#         deployment=embedding_llm_deployment_name,
#         openai_api_key= openai.api_key,
#         openai_api_base=openai.api_base,
#         openai_api_type=openai.api_type,
#         openai_api_version=openai.api_version,
#     ),
#     embed_batch_size=1,
# )

service_context = ServiceContext.from_defaults(
    node_parser=node_parser,
    # embed_model=embedding_llm,
    llm_predictor=llm_predictor_gpt3
)

service_context_gpt4 = ServiceContext.from_defaults(
    node_parser=node_parser,
    # embed_model=embedding_llm,
    llm_predictor=llm_predictor_gpt4
)

In [9]:
from qdrant_client import QdrantClient
import logging
import sys

# Qdrantクライアントを用意
client = QdrantClient(host='qdrant', port=6333)
print("Connection successful:", client)

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

Connection successful: <qdrant_client.qdrant_client.QdrantClient object at 0x7efcbf7dd690>


In [10]:
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

# VectorStoreIndexをdocumentsから構築
vector_store = QdrantVectorStore(client=client, collection_name="test_rag2")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

INFO:httpx:HTTP Request: GET http://qdrant:6333/collections/test_rag2 "HTTP/1.1 404 Not Found"
HTTP Request: GET http://qdrant:6333/collections/test_rag2 "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: DELETE http://qdrant:6333/collections/test_rag2 "HTTP/1.1 200 OK"
HTTP Request: DELETE http://qdrant:6333/collections/test_rag2 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://qdrant:6333/collections/test_rag2 "HTTP/1.1 200 OK"
HTTP Request: PUT http://qdrant:6333/collections/test_rag2 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://qdrant:6333/collections/test_rag2/points?wait=true "HTTP/1.1 200 OK"
HTTP Request: PUT http://qdrant:6333/collections/test_rag2/points?wait=true "HTTP/1.1 200 OK"


In [11]:
query_engine = index.as_query_engine()
# gpt4を使う場合
# query_engine = index.as_query_engine(service_context=service_context_gpt4)

In [12]:
response = query_engine.query("提案手法の概要を説明してください")
print(response)

INFO:httpx:HTTP Request: POST http://qdrant:6333/collections/test_rag2/points/search "HTTP/1.1 200 OK"
HTTP Request: POST http://qdrant:6333/collections/test_rag2/points/search "HTTP/1.1 200 OK"
提案手法は、画像加工を組み合わせることによって、異常検知精度の維持と高い圧縮率の両立を実現する手法です。具体的には、各画像の切り抜き領域を示すマスク画像を作成し、オブジェクトに干渉しない範囲の背景を大まかに削除します。その後、加工した画像データセットを動画ファイルに変換し、HEVCの非可逆圧縮モードを用いて圧縮します。HEVCによる変換では、画像間の冗長性を排除して符号化されるため、画像データセットのサイズが大幅に縮小されます。そして、モデルの学習時には、動画ファイルから個々の画像ファイルに復元して学習処理を行います。このように、提案手法は画像加工と動画ファイルへの変換を組み合わせることで、異常検知モデルの学習用データセットを効果的に圧縮することができます。


# ベクターストアからの復元(失敗)

In [15]:
vector_store2 = QdrantVectorStore(client=client, collection_name="test_rag2")
storage_context2 = StorageContext.from_defaults(vector_store=vector_store2)
index2 = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context2, service_context=service_context
)

INFO:httpx:HTTP Request: GET http://qdrant:6333/collections/test_rag2 "HTTP/1.1 200 OK"
HTTP Request: GET http://qdrant:6333/collections/test_rag2 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://qdrant:6333/collections/test_rag2/points?wait=true "HTTP/1.1 200 OK"
HTTP Request: PUT http://qdrant:6333/collections/test_rag2/points?wait=true "HTTP/1.1 200 OK"


In [16]:
query_engine2 = index2.as_query_engine()

In [17]:
response2 = query_engine2.query("提案手法の概要を説明してください")
print(response2)

INFO:httpx:HTTP Request: POST http://qdrant:6333/collections/test_rag2/points/search "HTTP/1.1 200 OK"
HTTP Request: POST http://qdrant:6333/collections/test_rag2/points/search "HTTP/1.1 200 OK"
提案手法は、画像加工による異常検知モデル学習用データセットの圧縮手法です。まず、各画像の切り抜き領域を示すマスク画像の論理和を求め、すべてのオブジェクトが収まる最小領域を持ったマスク画像を作成します。そして、このマスク画像を使用して、全ての画像に対してマスク処理を行い、オブジェクトに干渉しない範囲の背景を大まかに削除します。その後、加工した画像データセットをHEVCの非可逆圧縮モードを用いて動画ファイルに変換します。HEVCによる変換では、画像間の冗長性を排除して符号化されるため、画像データセットサイズの大幅な縮小が期待されます。最後に、モデルの学習時には、一度動画ファイルへと変換したデータセットを再び個々の画像ファイルに復元し、従来と同様の方法で学習処理を行います。


# ベクターストアから復元(成功？)
一応，embeddingは質問の1回だけしか行われなかった

In [24]:
index2.storage_context.persist(persist_dir="./storage_context")
# documentもqdrantに保存されていて，出力されるjsonはそのアドレスを示しているだけ？

In [20]:
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore

storage_context3 = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="./storage_context"),
    vector_store=vector_store2,
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="./storage_context"),
)

In [21]:
index3 = load_index_from_storage(storage_context3, service_context=service_context)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.


In [22]:
query_engine3 = index3.as_query_engine()

In [23]:
response3 = query_engine3.query("提案手法の概要を説明してください")
print(response3)

INFO:httpx:HTTP Request: POST http://qdrant:6333/collections/test_rag2/points/search "HTTP/1.1 200 OK"
HTTP Request: POST http://qdrant:6333/collections/test_rag2/points/search "HTTP/1.1 200 OK"
画像加工による異常検知モデル学習用データセットの圧縮手法の提案では、以下の手順でデータセットの圧縮を行っています。

まず、各画像の切り抜き領域を示すマスク画像の論理和を求め、すべてのオブジェクトが収まる最小領域を持ったマスク画像を作成します。これにより、オブジェクトに干渉しない範囲の背景を大まかに削除することができます。

次に、加工した画像データセットを動画ファイルに変換します。変換は、HEVCの非可逆圧縮モードを用いて行われ、画像1枚を動画データの1フレームとして圧縮します。HEVCによる変換では、画像間の冗長性を排除して符号化されるため、画像データセットのサイズが大幅に縮小されることが期待されます。

最後に、モデルの学習時には、一度動画ファイルへと変換したデータセットを再び個々の画像ファイルに復元します。これにより、従来と同様の方法で学習処理を行うことができます。
